In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from tqdm import tqdm_notebook
from itertools import product

import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#connnecting drive with google colab
from google.colab import drive
drive.mount('/content/drive/')
#read the csv file 
macro_data=pd.read_csv('/content/drive/My  Drive/Colab Notebooks/Project/Covid data set for VAR - Covid data set for VAR.csv', parse_dates= ['date'], index_col='date')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
print(macro_data.shape)  
macro_data.head()

(132851, 64)


,iso_code,continent,location,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,female_smokers,male_smokers,handwashing_facilities,handwashing_facilities.1,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
date,,,,,,,,,,,,,,,,,,,,,
2020-02-24,AFG,Asia,Afghanistan,5.0,5.0,NaN,NaN,NaN,NaN,0.126,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2020-02-25,AFG,Asia,Afghanistan,5.0,0.0,NaN,NaN,NaN,NaN,0.126,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2020-02-26,AFG,Asia,Afghanistan,5.0,0.0,NaN,NaN,NaN,NaN,0.126,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2020-02-27,AFG,Asia,Afghanistan,5.0,0.0,NaN,NaN,NaN,NaN,0.126,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2020-02-28,AFG,Asia,Afghanistan,5.0,0.0,NaN,NaN,NaN,NaN,0.126,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [ ]:

df1= macro_data.drop(['iso_code', 'continent', 'location','handwashing_facilities','gdp_per_capita','tests_units', 'female_smokers','male_smokers'], axis= 1)


df2= df1.ffill(axis= 0)
df3= df2.bfill(axis= 0)

In [ ]:
columns = ['total_cases','total_deaths','people_fully_vaccinated','new_cases']
rr = df3[columns]
rr.head()

,total_cases,total_deaths,people_fully_vaccinated,new_cases
date,,,,
2020-02-24,5.0,1.0,55624.0,5.0
2020-02-25,5.0,1.0,55624.0,0.0
2020-02-26,5.0,1.0,55624.0,0.0
2020-02-27,5.0,1.0,55624.0,0.0
2020-02-28,5.0,1.0,55624.0,0.0


Checking if data is stationary or not. If p value is less than 0.05 than data is stationary otherwise data is not stationary. By doing 1st order differencing we are making the data stationary.

In [ ]:
ad_fuller_result_1 = adfuller(rr['total_cases'].diff()[1:])

print('total_cases')
print(f'ADF Statistic: {ad_fuller_result_1[0]}')
print(f'p-value: {ad_fuller_result_1[1]}')

print('\n---------------------\n')

ad_fuller_result_2 = adfuller(rr['people_fully_vaccinated'].diff()[1:])

print('people_fully_vaccinated')
print(f'ADF Statistic: {ad_fuller_result_2[0]}')
print(f'p-value: {ad_fuller_result_2[1]}')

total_cases
ADF Statistic: -364.40195727834936
p-value: 0.0

---------------------

people_fully_vaccinated
ADF Statistic: -68.47784156576859
p-value: 0.0


Granger Causality test is used to check correlation between the two time series. Tells whether the time series in the second column causes the time series in the first column. Currently doing for 4 lags but we can do for multiple lags. If p=0 means the second column causes the time series in the first column.

In [ ]:
print(' people_fully_vaccinated causes total_cases?\n')
print('------------------')
granger_1 = grangercausalitytests(rr[['people_fully_vaccinated', 'total_cases']], 4)

print('\n people_fully_vaccinated causes total_cases?\n')
print('------------------')
granger_2 = grangercausalitytests(rr[['total_cases', 'people_fully_vaccinated']], 4)

 people_fully_vaccinated causes total_cases?

------------------

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=474.8355, p=0.0000  , df_denom=132847, df_num=1
ssr based chi2 test:   chi2=474.8462, p=0.0000  , df=1
likelihood ratio test: chi2=473.9996, p=0.0000  , df=1
parameter F test:         F=474.8355, p=0.0000  , df_denom=132847, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=236.6068, p=0.0000  , df_denom=132844, df_num=2
ssr based chi2 test:   chi2=473.2315, p=0.0000  , df=2
likelihood ratio test: chi2=472.3906, p=0.0000  , df=2
parameter F test:         F=236.6068, p=0.0000  , df_denom=132844, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=157.2040, p=0.0000  , df_denom=132841, df_num=3
ssr based chi2 test:   chi2=471.6367, p=0.0000  , df=3
likelihood ratio test: chi2=470.8015, p=0.0000  , df=3
parameter F test:         F=157.2040, p=0.0000  , df_denom=132841, df_num=3

Granger Ca

In [ ]:
tcount = int(len(rr)*0.7)

train = rr.iloc[0:tcount]
test= rr[tcount:int(len(rr))]

dataset = train
dataset

,total_cases,total_deaths,people_fully_vaccinated,new_cases
date,,,,
2020-02-24,5.0,1.0,55624.0,5.0
2020-02-25,5.0,1.0,55624.0,0.0
2020-02-26,5.0,1.0,55624.0,0.0
2020-02-27,5.0,1.0,55624.0,0.0
2020-02-28,5.0,1.0,55624.0,0.0
...,...,...,...,...
2021-01-10,279196.0,4455.0,1292573.0,2424.0
2021-01-11,281353.0,4500.0,1292573.0,2157.0
2021-01-12,285093.0,4561.0,1292573.0,3740.0


In [ ]:
y='total_deaths'
max_lags=8

In [ ]:
for i in range(len(dataset.columns)-1):
  results=grangercausalitytests(dataset[[y,dataset.columns[i+1]]], max_lags, verbose=False)
  p_values=[round(results[i+1][0]['ssr_ftest'][1],4) for i in range(max_lags)]
  print('Column - {} : P_Values - {}'.format(dataset.columns[i+1],p_values))

Column - total_deaths : P_Values - [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Column - people_fully_vaccinated : P_Values - [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Column - new_cases : P_Values - [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
df_train = dataset[:int(0.9*(len(dataset)))]
df_test = dataset[int(0.9*(len(dataset))):]

In [ ]:
df_train.shape
df_test.shape

(9300, 4)

In [ ]:
model = VAR(df_train.diff()[1:])

Select order help us to select the maximum lags. For a good model all the parameters should be minimum as possible. Normally only looking at AIC score is also sufficient. Star highlight where minimum is found. We need to consider that lag where minimum was found was further processing.

In [ ]:
sorted_order=model.select_order(maxlags=15)
print(sorted_order.summary())

 VAR Order Selection (* highlights the minimums)  
       AIC         BIC         FPE         HQIC   
--------------------------------------------------
0        94.88       94.88   1.600e+41       94.88
1        94.83       94.83   1.532e+41       94.83
2        94.79       94.79   1.466e+41       94.79
3        94.75       94.76   1.411e+41       94.75
4        94.71       94.72   1.356e+41       94.71
5        94.56       94.57   1.166e+41       94.56
6        94.43       94.44   1.023e+41       94.43
7        94.22       94.24   8.328e+40       94.23
8        94.09       94.11   7.322e+40       94.10
9        94.06       94.08   7.097e+40       94.07
10       94.05       94.07   7.036e+40       94.06
11       94.05       94.07   7.037e+40       94.06
12       94.05       94.08   7.038e+40       94.06
13       94.03       94.05   6.840e+40       94.03
14       93.99       94.02   6.626e+40       94.00
15      93.83*      93.86*  5.627e+40*      93.84*
-------------------------------

VARMAX Class make forecasting easier. We are providing the non stationary data and not difference data. Order(Autoregression,Moving Average part).Enforce stationarity will make are data stationary.

In [ ]:
var_model = VAR(df_train)
fitted_model = var_model.fit(47)
print(fitted_model.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 07, Apr, 2022
Time:                     07:55:30
--------------------------------------------------------------------
No. of Equations:         4.00000    BIC:                    93.7130
Nobs:                     83648.0    HQIC:                   93.6544
Log likelihood:      -4.38993e+06    FPE:                4.59600e+40
AIC:                      93.6286    Det(Omega_mle):     4.55469e+40
--------------------------------------------------------------------
Results for equation total_cases
                                 coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------------------------
const                            1591.563812      2167.719540            0.734           0.463
L1.total_cases                      0.973054         0.003604          270.005           0.00

In [ ]:
predict = fitted_model.forecast(start=len(df_train),end=len(df_train) +len(df_test)-1)
predictions=predict.predicted_mean

TypeError: ignored

In [ ]:
predictions.columns=['Predicted_total_cases',	'Predicted_total_deaths',	'Predicted_people_fully_vaccinated',	'Predicted_new_cases']
predictions.head()

,Predicted_total_cases,Predicted_total_deaths,Predicted_people_fully_vaccinated,Predicted_new_cases
83695,131041.684727,3684.174482,271018.790222,77.536407
83696,133396.905662,3802.138008,268290.756527,127.116367
83697,135906.618490,3917.715602,266005.202531,165.940719
83698,138290.221395,4030.707845,263268.761053,199.272113
83699,140700.980530,4142.353559,259997.210749,239.520259


In [ ]:
# Common code for display result
def show_graph(df1,df2,title):
    data = pd.concat([df1, df2])
    data.reset_index(inplace=True, drop=True)
    for col in data.columns:
        if col.lower().endswith('_forecast'):
            data[col].plot(label=col,linestyle="dashed")
        else:
            data[col].plot(label=col)
    plt.title(title)
    plt.legend()
    plt.show()

,total_cases,total_deaths,people_fully_vaccinated,new_cases
date,,,,
2020-02-24,5.0,1.0,55624.0,5.0
2020-02-25,5.0,1.0,55624.0,0.0
2020-02-26,5.0,1.0,55624.0,0.0
2020-02-27,5.0,1.0,55624.0,0.0
2020-02-28,5.0,1.0,55624.0,0.0


In [ ]:
from sklearn.metrics import mean_squared_error
import math 
from statistics import mean

rmse_cases=math.sqrt(mean_squared_error(predictions['Predicted_total_cases'],df_test['total_cases']))
print('Mean value of Cases is : {}. Root Mean Squared Error is :{}'.format(mean(df_test['total_cases']),rmse_cases))

rmse_deaths=math.sqrt(mean_squared_error(predictions['Predicted_total_deaths'],df_test['total_deaths']))
print('Mean value of deaths is : {}. Root Mean Squared Error is :{}'.format(mean(df_test['total_deaths']),rmse_deaths))

rmse_people_fully_vaccinated=math.sqrt(mean_squared_error(predictions['Predicted_people_fully_vaccinated'],df_test['people_fully_vaccinated']))
print('Mean value of  people_fully_vaccinated is : {}. Root Mean Squared Error is :{}'.format(mean(df_test['people_fully_vaccinated']),rmse_people_fully_vaccinated))

rmse_new_cases=math.sqrt(mean_squared_error(predictions['Predicted_new_cases'],df_test['new_cases']))
print('Mean value of  is : {}. Root Mean Squared Error is :{}'.format(mean(df_test['new_cases']),rmse_new_cases))




Mean value of Cases is : 1829096.5753763441. Root Mean Squared Error is :7552444.308331592
Mean value of deaths is : 43839.46526881721. Root Mean Squared Error is :174988.41932269954
Mean value of  people_fully_vaccinated is : 25236093.488817204. Root Mean Squared Error is :74041451.91824196
Mean value of  is : 7257.111720430107. Root Mean Squared Error is :29363.32627011895
